In [25]:
import pandas as pd

In [26]:
def trns(value):
    s = value.split("<code>")[1].split("</code>")[0]
    return s

In [45]:
def get_question_answer_pair(df):
    idx = []
    for i in range(len(df)):
        if df.Answer[i].count('<code>def ') == 1 and df.Answer[i].count('<code>') == 1:
            idx.append(i)

    df_qa = pd.DataFrame({
        "View_Count": df.loc[idx]['ViewCount'],
        "Question_Title": df.loc[idx]['Question Title'], 
        "Answer": df.loc[idx].Answer.apply(trns),
        "Tags": df.loc[idx]['Tags']
        })
    return df_qa

### Data with no filter

In [46]:
df = pd.read_csv('QueryResults.csv')
df.head()

,Post Link,Question Score,ViewCount,Question Title,Question Content,Answer,Tags
0,48174398,4,435,New Dataframe column as a generic function of ...,<p><strong>What is the fastest (and most effic...,<p>Let's try to analyze the problem for a seco...,<python><pandas><dataframe><vectorization>
1,48211001,0,56,Python: How to update multiple address lines,<p>I'm currently working on a banking system f...,<p>Your issue is arguably in your <code>update...,<python>
2,48213278,1,2389,Implementing Otsu binarization from scratch py...,<p>It seems my implementation is incorrect and...,<p>I dont know if my implementation is alright...,<python><python-3.x><image-processing><compute...
3,48217471,2,311,Is it possible to check for anagram without us...,<p>I'm currently studying and I was told not t...,<p>The pythonic way of answering this question...,<python><algorithm>
4,48217583,1,531,Accessing attributes of user defined object le...,<p>I have a class called LineString that consi...,<p>The cause for your error is the fact that y...,<python><class><typeerror>


In [47]:
df.shape

(30510, 7)

In [53]:
df_without_filter = get_question_answer_pair(df)
len(df_without_filter)

2012

In [49]:
#df_qa.sort_values(by = 'View_Count', ascending=False)
df_without_filter.head()

,View_Count,Question_Title,Answer,Tags
2,2389,Implementing Otsu binarization from scratch py...,def otsu(gray):\n pixel_number = gray.shape...,<python><python-3.x><image-processing><compute...
8,51,Replacing elements from a string,"def replace_chars(tmpStr, tmpChar):\n for i...",<python><python-3.x>
9,243,Refreshing the page and check for text every 1...,def foo():\n src = driver.page_source\n ...,<python-3.x><selenium-webdriver><pycharm>
10,52,Python Tkinter loop in a loop?,def turnOn():\n global proc\n if proc is No...,<python><tkinter>
25,59,Django. Comparing the model fields before savi...,"def save(self, *args, **kwargs):\n super(De...",<python><django><django-models>


### Data with filter (Python Question Only)

In [58]:
#df[df['Tags'].str.contains('django|pyspark|flask|django|pandas|keras|tensorflow|pytorch|sklearn|opencv|numpy|pyqt|pygame') != True]
df_python = df[((df['Tags'] == '<python>'))|(df['Tags'] == '<python-3.x>')].reset_index()
df_with_filter = get_question_answer_pair(df_python)
len(df_with_filter)

224

In [59]:
df_with_filter.head()

,View_Count,Question_Title,Answer,Tags
12,43,Add to dictionary specified item,def process_user_input():\n valid_options =...,<python>
14,27,why my python program is printing none on next...,"def rotatelist(l, k):\n n = len(l)\n if ...",<python-3.x>
24,23,python dela with variable in memory,"def choice(n,s=None):\n if s is None:\n ...",<python>
33,53,The code will not run nor will it show an erro...,"def main():\n s= """"\n phrase=""""\n pro...",<python>
50,40,Python CSV to Table in Email,"def create_table():\n message = """"\n mes...",<python>


### Example

In [51]:
# Example Question
idx = 10
df_with_filter.Question_Title[idx]

'Add to dictionary specified item'

In [52]:
# Example Answer
df_with_filter.Answer[idx].split("\n")

['def process_user_input():',
 "    valid_options = {'blue':20, 'pink':12, 'red':18, 'green':10, 'black':15, 'yellow':6};",
 '    choosen_item  = {};',
 '',
 "    user_input = raw_input('search for a colour: ');",
 '',
 '    for key, value in valid_options.iteritems():',
 '        if int(user_input) == value:',
 '            choosen_item[key] = value;',
 '',
 '    print(choosen_item);',
 '',
 'process_user_input();',
 '',
 '# search for a colour:  20',
 "# {'blue': 20}",
 '']

In [ ]:
#### SQL For StackOverFlow

SELECT Posts.Id as [Post Link], 
Posts.Score as [Question Score],
Posts.ViewCount,
Posts.Title As [Question Title],
Posts.Body As [Question Content],
Answers.Body AS [Answer],
Answers.ViewCount


From Posts
left join Posts As Answers 
        ON Answers.parentid = Posts.id
        AND Answers.posttypeid = 2

WHERE Posts.PostTypeId = 1 AND 
YEAR(Posts.CreationDate) BETWEEN 2018 AND 2019 AND
Posts.AcceptedAnswerId = Answers.Id AND
Posts.Tags LIKE '%python%' AND
Posts.Tags NOT LIKE '%pandas%' AND
Posts.Tags NOT LIKE '%pyspark%' AND
Posts.Tags NOT LIKE '%flask%' AND
Posts.Tags NOT LIKE '%django%' AND
Posts.Tags NOT LIKE '%keras%' AND
Posts.Tags NOT LIKE '%tensorflow%' AND
Posts.Tags NOT LIKE '%pytorch%' AND
Posts.Tags NOT LIKE '%sklearn%' AND
Posts.Tags NOT LIKE '%opencv%' AND
Posts.Tags NOT LIKE '%numpy%' AND
Posts.Tags NOT LIKE '%pyqt%' AND
Posts.Tags NOT LIKE '%pygame%' 

GROUP BY
Posts.Id, Posts.Score, Posts.ViewCount, Posts.Title, Posts.ViewCount, Answers.Body, Posts.Body, Posts.AcceptedAnswerId, Answers.Id, Answers.ViewCount

HAVING Answers.Body LIKE '%def %'

